In [4]:
class FakeArgs:
    def __init__(self, red_codemaster, red_guesser, blue_codemaster, blue_guesser):
        self.red_codemaster = red_codemaster
        self.red_guesser = red_guesser
        self.blue_codemaster = blue_codemaster
        self.blue_guesser = blue_guesser
        self.seed = "time"
        self.w2v = "players/GoogleNews-vectors-negative300.bin"
        self.glove = None
        self.wordnet = None
        self.glove_cm = "players/glove.6B.100d.txt"
        self.glove_guesser = "players/glove.6B.100d.txt"
        self.no_log = None
        self.no_print = None
        self.game_name = "default"

In [5]:
import sys
import importlib
import argparse
import time
import os

from game import Game
from players.guesser import *
from players.codemaster import *

TWO_PLAYER = True

class GameRun:
    """Class that builds and runs a Game based on command line arguments"""

    def __init__(self):
        #parser = argparse.ArgumentParser(
            #description="Run the Codenames AI competition game.",
            #formatter_class=argparse.ArgumentDefaultsHelpFormatter)
        #if TWO_PLAYER:
        #    parser.add_argument("red_codemaster", help="import string of form A.B.C.MyClass or 'human'")
        #    parser.add_argument("red_guesser", help="import string of form A.B.C.MyClass or 'human'")
        #    parser.add_argument("blue_codemaster", help="import string of form A.B.C.MyClass or 'human'")
        #    parser.add_argument("blue_guesser", help="import string of form A.B.C.MyClass or 'human'")
        #parser.add_argument("--seed", help="Random seed value for board state -- integer or 'time'", default='time')

        #parser.add_argument("--w2v", help="Path to w2v file or None", default=None)
        #parser.add_argument("--glove", help="Path to glove file or None", default=None)
        #parser.add_argument("--wordnet", help="Name of wordnet file or None, most like ic-brown.dat", default=None)
        #parser.add_argument("--glove_cm", help="Path to glove file or None", default=None)
        #parser.add_argument("--glove_guesser", help="Path to glove file or None", default=None)

        #parser.add_argument("--no_log", help="Supress logging", action='store_true', default=False)
        #parser.add_argument("--no_print", help="Supress printing", action='store_true', default=False)
        #parser.add_argument("--game_name", help="Name of game in log", default="default")

        #args = parser.parse_args()
        args = FakeArgs("players.codemaster_glove_07_adversary.AICodemaster",\
                        "players.guesser_glove.AIGuesser",\
                        "players.codemaster_glove_07.AICodemaster",\
                        "players.guesser_glove.AIGuesser")
        self.do_log = not args.no_log
        self.do_print = not args.no_print
        if not self.do_print:
            self._save_stdout = sys.stdout
            sys.stdout = open(os.devnull, 'w')
        self.game_name = args.game_name

        self.g_kwargs = {}
        self.cm_kwargs = {}

        # load codemaster class
        if TWO_PLAYER:
            self.red_codemaster = self.import_string_to_class(args.red_codemaster)
            self.red_guesser = self.import_string_to_class(args.red_guesser)
            self.blue_codemaster = self.import_string_to_class(args.blue_codemaster)
            self.blue_guesser = self.import_string_to_class(args.blue_guesser)
            print(f'Red CM: {args.red_codemaster} BLUE CM: {args.blue_codemaster}')
            print(f'RED G: {args.red_guesser} BLUE G {args.blue_guesser}')
        else:
            if args.codemaster == "human":
                self.codemaster = HumanCodemaster
                print('human codemaster')
            else:
                self.codemaster = self.import_string_to_class(args.codemaster)
                print('loaded codemaster class')

            # load guesser class
            if args.guesser == "human":
                self.guesser = HumanGuesser
                print('human guesser')
            else:
                self.guesser = self.import_string_to_class(args.guesser)
                print('loaded guesser class')

        # if the game is going to have an ai, load up word vectors
        if sys.argv[1] != "human" or sys.argv[2] != "human":
            if args.wordnet is not None:
                brown_ic = Game.load_wordnet(args.wordnet)
                self.g_kwargs["brown_ic"] = brown_ic
                self.cm_kwargs["brown_ic"] = brown_ic
                print('loaded wordnet')

            if args.glove is not None:
                glove_vectors = Game.load_glove_vecs(args.glove)
                self.g_kwargs["glove_vecs"] = glove_vectors
                self.cm_kwargs["glove_vecs"] = glove_vectors
                print('loaded glove vectors')

            if args.w2v is not None:
                w2v_vectors = Game.load_w2v(args.w2v)
                self.g_kwargs["word_vectors"] = w2v_vectors
                self.cm_kwargs["word_vectors"] = w2v_vectors
                print('loaded word vectors')

            if args.glove_cm is not None:
                glove_vectors = Game.load_glove_vecs(args.glove_cm)
                self.cm_kwargs["glove_vecs"] = glove_vectors
                print('loaded glove vectors')

            if args.glove_guesser is not None:
                glove_vectors = Game.load_glove_vecs(args.glove_guesser)
                self.g_kwargs["glove_vecs"] = glove_vectors
                print('loaded glove vectors')

        # set seed so that board/keygrid can be reloaded later
        if args.seed == 'time':
            self.seed = time.time()
        else:
            self.seed = int(args.seed)

    def __del__(self):
        """reset stdout if using the do_print==False option"""
        if not self.do_print:
            sys.stdout.close()
            sys.stdout = self._save_stdout

    def import_string_to_class(self, import_string):
        """Parse an import string and return the class"""
        parts = import_string.split('.')
        module_name = '.'.join(parts[:len(parts) - 1])
        class_name = parts[-1]

        module = importlib.import_module(module_name)
        my_class = getattr(module, class_name)

        return my_class


if __name__ == "__main__":
    game_setup = GameRun()
    game = Game(game_setup.red_codemaster,
                game_setup.red_guesser,
                game_setup.blue_codemaster,
                game_setup.blue_guesser,
                seed=game_setup.seed,
                do_print=game_setup.do_print,
                do_log=game_setup.do_log,
                game_name=game_setup.game_name,
                cm_kwargs=game_setup.cm_kwargs,
                g_kwargs=game_setup.g_kwargs)

    game.run()

Red CM: players.codemaster_glove_07_adversary.AICodemaster BLUE CM: players.codemaster_glove_07.AICodemaster
RED G: players.guesser_glove.AIGuesser BLUE G players.guesser_glove.AIGuesser
loaded word vectors
loaded glove vectors
loaded glove vectors
seed: 1638757246.671965





____________________________KEY____________________________

    Red         Red      Assassin       Red      Civilian   

   Blue         Red        Blue        Blue        Blue     

 Civilian    Civilian       Red      Civilian    Civilian   

   Blue      Civilian    Civilian    Civilian      Blue     

    Red      Civilian       Red        Blue         Red     
___________________________________________________________


___________________________BOARD___________________________

   SPELL       INDIA       CHEST       BOLT        THUMB    

   POLE        PUPIL      FRANCE       ROCK       CANADA    

   PARK        LASER       WALL      HOSPITAL      PITCH    

   PLANE       EAGLE    HELICOPTER     GLAS

guesses: [(0.4890727984999035, 'POLE'), (0.4904281422250707, 'FRANCE'), (0.5047522039209188, 'PITCH'), (0.5569680463331359, 'CROWN'), (0.5671741577730546, 'INDIA'), (0.6031391140045531, 'EAGLE'), (0.618224664412093, 'PARK'), (0.6226006775825023, 'PLANE'), (0.6349799783015775, 'HOSPITAL'), (0.6363065713976036, 'SPELL'), (0.6500804745997775, 'ROCK'), (0.7095053460425407, 'GLASS'), (0.712871529282513, 'HELICOPTER'), (0.7338953039626064, 'BEACH'), (0.737119439027589, 'CHEST'), (0.7448572949697105, 'BOLT'), (0.7458708307333131, 'COTTON'), (0.7577959781431147, 'PUPIL'), (0.7884295169932225, 'THUMB'), (0.8419249570615455, 'LASER'), (1.0355953293288767, 'OCTOPUS')]



___________________________BOARD___________________________

   SPELL       INDIA       CHEST       BOLT        THUMB    

  *Blue*       PUPIL      FRANCE       ROCK       *Blue*    

   PARK        LASER       *Red*     HOSPITAL      PITCH    

   PLANE       EAGLE    HELICOPTER     GLASS      COTTON    

   *Red*      OCTOPUS 

guesses: [(0.5228235165376954, 'PLANE'), (0.5669605117764911, 'ROCK'), (0.5846914701903634, 'PARK'), (0.6144966531105529, 'HELICOPTER'), (0.6774558168706233, 'GLASS'), (0.7111380188043589, 'EAGLE'), (0.7521915364658684, 'OCTOPUS'), (0.754714601333574, 'PITCH'), (0.7737356997817684, 'SPELL'), (0.8128502428571254, 'LASER'), (0.817995276451565, 'CHEST'), (0.8213704725282842, 'HOSPITAL'), (0.8613594026174828, 'COTTON'), (0.907575584237615, 'BOLT'), (0.9511321179094588, 'THUMB'), (1.02210698201459, 'PUPIL')]



___________________________BOARD___________________________

   SPELL       *Red*       CHEST       BOLT        THUMB    

  *Blue*       PUPIL      *Blue*       ROCK       *Blue*    

   PARK        LASER       *Red*     HOSPITAL      PITCH    

  *Blue*       EAGLE    HELICOPTER     GLASS      COTTON    

   *Red*      OCTOPUS      *Red*      *Blue*       *Red*    
___________________________________________________________


Keep Guessing? the clue is  ocean 3
guesses: [(0.5669605